In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import logging

logger = logging.getLogger('')
logger.setLevel(logging.INFO)


In [6]:
# 假设 client_nums = 5, participants_nums = 3
# [1, 1, 1, 0, 0, 0.66, 1, 0, 1, 0, 1, 0.70]
# action = 0, 2, 3
# [1, 0, 1, 1, 0, 0.77] -> [action + reward]
# [1, 0, 1, 0, 1, 0.70, 1, 0, 1, 1, 0, 0.77] - > update obs

class Model2D(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.obs_dim = obs_dim
        self.act_dim = act_dim

        self.body_1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc1 = nn.Linear(32 * 101 * 1, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 100)

        # self.body_2 = nn.Sequential(
        #     nn.Linear(32 * 101 * 2, 128),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(),
        #     nn.Linear(128, 100),
        # )
    def forward(self, obs):
        x = self.body_1(obs)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.fc2(x)
        # x = self.body_2(obs)
        return F.softmax(x, dim=1)

In [7]:
model = Model2D(1,1)

In [15]:
x = torch.randn([4, 404])
x.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 4, 404])

In [9]:
x = torch.randn([1,1,4,404])

In [11]:
model.eval()
model(x)

tensor([[0.0104, 0.0104, 0.0101, 0.0110, 0.0094, 0.0096, 0.0096, 0.0105, 0.0104,
         0.0097, 0.0083, 0.0086, 0.0093, 0.0099, 0.0109, 0.0099, 0.0095, 0.0090,
         0.0118, 0.0102, 0.0102, 0.0096, 0.0105, 0.0096, 0.0094, 0.0097, 0.0102,
         0.0102, 0.0107, 0.0108, 0.0102, 0.0096, 0.0105, 0.0089, 0.0092, 0.0101,
         0.0118, 0.0100, 0.0106, 0.0104, 0.0105, 0.0089, 0.0112, 0.0106, 0.0114,
         0.0094, 0.0101, 0.0106, 0.0115, 0.0111, 0.0111, 0.0093, 0.0103, 0.0093,
         0.0102, 0.0090, 0.0101, 0.0106, 0.0103, 0.0093, 0.0102, 0.0104, 0.0105,
         0.0098, 0.0088, 0.0097, 0.0095, 0.0118, 0.0098, 0.0125, 0.0094, 0.0095,
         0.0096, 0.0094, 0.0104, 0.0094, 0.0099, 0.0099, 0.0098, 0.0097, 0.0104,
         0.0096, 0.0100, 0.0084, 0.0088, 0.0090, 0.0093, 0.0090, 0.0095, 0.0101,
         0.0108, 0.0091, 0.0084, 0.0109, 0.0107, 0.0103, 0.0103, 0.0103, 0.0090,
         0.0108]], grad_fn=<SoftmaxBackward0>)